In [ ]:
import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import matplotlib
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

from albumentations import (
    Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, Rotate, ShiftScaleRotate, Cutout, 
    IAAAdditiveGaussianNoise, Transpose
    )
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda:'+str(0) if torch.cuda.is_available() else 'cpu')

In [ ]:
train_img_path = os.listdir('../input/cassava-leaf-disease-classification/train_images')
print(len(train_img_path))
data_path = '../input/cassava-leaf-disease-classification'

In [ ]:
test = pd.read_csv(data_path+'/sample_submission.csv')
label_map = pd.read_json(data_path+'/label_num_to_disease_map.json', orient='index')

In [ ]:
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['image_id'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = data_path+'/test_images/'+str(file_name)
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image

In [ ]:
def get_transforms(*, data):
    if data == 'test':
        return Compose([
            Resize(380, 380),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

In [ ]:
class BottleNeck(nn.Module):
    expansion = 2

    def __init__(self, in_channel, channel, stride=1, C=32, downsample=None):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channel, channel, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(channel),
            nn.ReLU(inplace=True)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(channel, channel, kernel_size=3, padding=1, bias=False, stride=1, groups=C),
            nn.BatchNorm2d(channel),
            nn.ReLU(inplace=True)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(channel, channel * self.expansion, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(channel * self.expansion),
            nn.ReLU(inplace=True)
        )

        self.downsample = downsample
        self.stride = stride
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = x
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        # out = self.relu(self.bn1(self.conv1(x)))  # bs,c,h,w
        # out = self.relu(self.bn2(self.conv2(out)))  # bs,c,h,w
        # out = self.relu(self.bn3(self.conv3(out)))  # bs,4c,h,w

        if self.downsample is not None:
            residual = self.downsample(residual)

        x = x + residual
        x = self.relu(x)
        return x


class ResNeXt(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super().__init__()
        # 定义输入模块的维度
        self.in_channel = 64
        # stem layer
        self.layer0 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1, ceil_mode=False)
        )

        # main layer
        self.layer1 = self._make_layer(block, 128, layers[0])
        self.layer2 = self._make_layer(block, 256, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 512, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 1024, layers[3], stride=2)

        # classifier
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(1024 * block.expansion, num_classes)
        self.downsample = None

    def forward(self, x):
        # stem layer
        x = self.layer0(x)

        # layers:
        x = self.layer1(x)  # bs,56,56,128*2
        x = self.layer2(x)  # bs,28,28,256*2
        x = self.layer3(x)  # bs,14,14,512*2
        x = self.layer4(x)  # bs,7,7,1024*2

        # classifier
        x = self.avgpool(x)  # bs,1,1,1024*2
        x = x.reshape(x.shape[0], -1)  # bs,1024*2
        x = self.classifier(x)  # bs,1000

        return x

    def _make_layer(self, block, channel, blocks, stride=1):
        # downsample 主要用来处理H(x)=F(x)+x中F(x)和x的channel维度不匹配问题，即对残差结构的输入进行升维，在做残差相加的时候，必须保证残差的纬度与真正的输出维度（宽、高、以及深度）相同
        # 比如步长！=1 或者 in_channel!=channel&self.expansion
        downsample = None
        if stride != 1 or self.in_channel != channel * block.expansion:
            self.downsample = nn.Conv2d(self.in_channel, channel * block.expansion, stride=stride, kernel_size=1,
                                        bias=False)
        # 第一个conv部分，可能需要downsample
        layers = list()
        layers.append(block(self.in_channel, channel, downsample=self.downsample, stride=stride))
        self.in_channel = channel * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channel, channel))
        return nn.Sequential(*layers)
    
    
def ResNeXt50(num_classes=1000):
    return ResNeXt(BottleNeck, [3, 4, 6, 3], num_classes=num_classes)

In [ ]:
chectpoint = torch.load('../input/resnext/resnext_0.pth', map_location='cuda:0')

pred=[]
test = pd.read_csv(data_path+'/sample_submission.csv')
model = ResNeXt50(5)
model.load_state_dict(chectpoint['model'])

test_dataset = TestDataset(test, transform=get_transforms(data='test'))
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

def preds(model, dataloader, device):
    model.eval()
    model.to(device)
    pred_list = []
    with torch.no_grad():
        tqbm_batch_iter = tqdm(dataloader)
        for image in tqbm_batch_iter:
            image = image.to(device)
            outputs = model(image)

            pred_y = torch.max(outputs, dim=1)[1]
            pred_list.extend(outputs.argmax(dim=1).tolist())
    return pred_list

# test_bar = tqdm(test_loader)
# for test_data in test_bar:
#     outputs = model(test_data.to(device))
#     predict_y = torch.max(outputs, dim=1)[1]
#     pred.append(int(predict_y))
# for image_id in test.image_id:
#     image = Image.open(os.path.join("../input/cassava-leaf-disease-classification",  "test_images", image_id))
#     image = image.resize((224,224))
#     image = np.expand_dims(image, axis = 0)
#     pred.append(torch.max(model(image.to(device), dim=1)[1]))
pred = preds(model, test_loader, device)
pred

In [ ]:
test['label'] = pred
test
test.to_csv('submission.csv', index = False)

In [ ]:
test.head()